In [1]:
import pandas as pd
import numpy as np
import joblib

# Sklearn imports
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV

class QuantileClipper(BaseEstimator, TransformerMixin):
    def __init__(self, lower=0.005, upper=0.995):
        self.lower = lower
        self.upper = upper
        self.q_low_ = None
        self.q_high_ = None

    def fit(self, X, y=None):
        # use np.nanpercentile to ignore NaN
        self.q_low_  = np.nanpercentile(X, self.lower * 100, axis=0)
        self.q_high_ = np.nanpercentile(X, self.upper * 100, axis=0)
        return self

    def transform(self, X):
        # use np.clip to clip data
        return np.clip(X, self.q_low_, self.q_high_)

## Load Data

In [2]:
file_path = 'data/final_df.csv' 
try:
    df_ori = pd.read_csv(file_path)
except FileNotFoundError:
    print(f"*** ERROR: Cannot find {file_path} ***")


In [3]:
df = df_ori.copy()

# Sort by symbol and datetime
df['datetime'] = pd.to_datetime(df['datetime'])
df = df.sort_values(by=['symbol', 'datetime'])
df = df.reset_index(drop=True)

df['clean_lret_1m'] = df.groupby('symbol')['close'].transform(lambda s: np.log(s).diff())
df['y_target'] = df.groupby('symbol')['clean_lret_1m'].shift(-1)

initial_rows = len(df)
df = df[(df['y_target'].abs() <= 0.2)]
df = df.dropna(subset=['y_target', 'clean_lret_1m'])

print(f"(Dropped {initial_rows - len(df)} unusal/NaN y value)")

# --- Re-sort by datatime, symbol ---
df = df.sort_values(by=['datetime', 'symbol'])
df = df.reset_index(drop=True)

df

(Dropped 12 unusal/NaN y value)


,datetime,symbol,dow_0,dow_1,dow_2,dow_3,dow_4,dow_5,f_minsin,f_mincos,...,split_nonpos_flag,shares_out,log_shares_out,eps_surp_pct_final,div_amount,log_shares_out_iqr_outlier,eps_estimate_rz_8,eps_actual,clean_lret_1m,y_target
0,2024-04-30 12:51:00,AMAT,0,1,0,0,0,0,-0.220697,-0.975342,...,1,830897024,20.538016,0.00,0.0,0,0.0,0.00,-0.000199,-0.000100
1,2024-04-30 12:51:00,AMD,0,1,0,0,0,0,-0.220697,-0.975342,...,1,1616140032,21.203306,2.04,0.0,0,0.0,0.62,0.000314,-0.000126
2,2024-04-30 12:51:00,AVGO,0,1,0,0,0,0,-0.220697,-0.975342,...,1,465308000,19.958210,0.00,0.0,0,0.0,0.00,-0.000334,0.000721
3,2024-04-30 12:51:00,MU,0,1,0,0,0,0,-0.220697,-0.975342,...,1,1107369984,20.825254,0.00,0.0,0,0.0,0.00,0.000698,-0.000698
4,2024-04-30 12:51:00,NVDA,0,1,0,0,0,0,-0.220697,-0.975342,...,1,2500000000,21.639557,0.00,0.0,0,0.0,0.00,-0.000769,-0.000907
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324883,2025-10-28 15:58:00,AMAT,0,1,0,0,0,0,-0.861629,-0.507538,...,1,796642427,20.495916,0.00,0.0,0,0.0,0.00,-0.000461,-0.000132
324884,2025-10-28 15:58:00,AMD,0,1,0,0,0,0,-0.861629,-0.507538,...,1,1633284837,21.213859,0.00,0.0,0,0.0,0.00,-0.000698,-0.001316
324885,2025-10-28 15:58:00,AVGO,0,1,0,0,0,0,-0.861629,-0.507538,...,1,4722365022,22.275576,0.00,0.0,0,0.0,0.00,0.000161,0.000375
324886,2025-10-28 15:58:00,MU,0,1,0,0,0,0,-0.861629,-0.507538,...,1,1122466035,20.838794,0.00,0.0,0,0.0,0.00,0.000270,-0.000135


In [4]:
df[["datetime", "symbol", "clean_lret_1m", "y_target"]]

,datetime,symbol,clean_lret_1m,y_target
0,2024-04-30 12:51:00,AMAT,-0.000199,-0.000100
1,2024-04-30 12:51:00,AMD,0.000314,-0.000126
2,2024-04-30 12:51:00,AVGO,-0.000334,0.000721
3,2024-04-30 12:51:00,MU,0.000698,-0.000698
4,2024-04-30 12:51:00,NVDA,-0.000769,-0.000907
...,...,...,...,...
324883,2025-10-28 15:58:00,AMAT,-0.000461,-0.000132
324884,2025-10-28 15:58:00,AMD,-0.000698,-0.001316
324885,2025-10-28 15:58:00,AVGO,0.000161,0.000375
324886,2025-10-28 15:58:00,MU,0.000270,-0.000135


## Data Cleaning

In [5]:
y = df['y_target']
X = df.drop(columns=[
    'y_target', 'clean_lret_1m', 'datetime', 'symbol', 
    'fz_lret_1_rolling', 
    'year', 'month', 'day', 'minute', 'minute_of_day'
], errors='ignore')

print(f"X original shape: {X.shape}")

# --- Find bad Cols with wrong data ---
desc = X.describe(percentiles=[0.99]).T
bad_cols = []

# std > 1000
bad_cols += desc.index[desc['std'] > 1e3].tolist()
# 99% > 1000
bad_cols += desc.index[desc['99%'].abs() > 1e3].tolist()
# Max > 1,000,000
bad_cols += desc.index[desc['max'].abs() > 1e6].tolist()
# Constant columns
bad_cols += desc.index[desc['std'] == 0].tolist()

# --- Manual add known bad col ---
known_bad_families = ['_rs', 'fz_lret_']
for col in X.columns:
    if any(family in col for family in known_bad_families):
        bad_cols.append(col)

bad_cols_set = sorted(set(bad_cols))
print(f"--- Dropping {len(bad_cols_set)} bad cols ---")
for col in bad_cols_set:
    print(f"  - {col}")

# --- Drop bad cols ---
X_cleaned = X.drop(columns=bad_cols_set)

print(f"\n--- X cleaned ---")
print(f"X (cleaned) shape: {X_cleaned.shape}")

X original shape: (324888, 161)
--- Dropping 32 bad cols ---
  - cnt_60m_rs
  - div_negative_flag
  - dow_5
  - dow_6
  - eps_actual_iqr_outlier
  - eps_estimate_iqr_outlier
  - fz_lret_15_rolling
  - fz_lret_30_rolling
  - fz_lret_5_rolling
  - fz_lret_60_rolling
  - fz_rsi_pos
  - mins_since_last_news
  - mins_since_last_news_rs
  - morning_n_rs
  - morning_source_div_rs
  - morning_tone_mean_rs
  - morning_tone_sum_rs
  - n_news_ewm_hl15_rs
  - n_news_ewm_hl5_rs
  - overnight_n_rs
  - overnight_source_div_rs
  - overnight_tone_mean_rs
  - overnight_tone_sum_rs
  - shares_out
  - shares_out_ffill
  - shares_out_nonpos_flag
  - split_flag
  - tone_ewm15_rs
  - tone_ewm30_rs
  - tone_mean_delta_rs
  - tone_mean_ewm_hl15_rs
  - volume

--- X cleaned ---
X (cleaned) shape: (324888, 129)


## Split Data

In [6]:
split_ratio = 1.0 / 1.5 
split_index = int(len(X_cleaned) * split_ratio)

X_val = X_cleaned.iloc[:split_index]
y_val = y.iloc[:split_index]

X_test = X_cleaned.iloc[split_index:]
y_test = y.iloc[split_index:]

print(f"--- Data Splitting done ---")
print(f" Validation set (X_val, y_val) shape: {X_val.shape}, {y_val.shape}")
print(f" Testing set (X_test, y_test) shape: {X_test.shape}, {y_test.shape}")

--- Data Splitting done ---
 Validation set (X_val, y_val) shape: (216592, 129), (216592,)
 Testing set (X_test, y_test) shape: (108296, 129), (108296,)


## Grid Search CV With MSE

In [7]:
## Pipeline
pipe = Pipeline([
    ("imp",  SimpleImputer(strategy="median")),
    ("clip", QuantileClipper(lower=0.005, upper=0.995)),
    ("sc",   MaxAbsScaler()),
    ("reg",  TransformedTargetRegressor(
                regressor=Ridge(solver="lsqr", max_iter=10000),
                transformer=StandardScaler())
    ),
])

# Hyper param grid
param_grid = {
    "reg__regressor__alpha": [10.0, 100.0, 1000.0, 1e4, 1e5]
}

# TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=10)

# Grid Search
print(f"\n--- GridSearchCV ... ---")
search = GridSearchCV(pipe, param_grid, 
                      cv=tscv,
                      scoring=["neg_mean_squared_error", 'r2'],
                      refit='neg_mean_squared_error',
                      n_jobs=-1, verbose=1)
search.fit(X_val, y_val)

# Result
best_alpha = search.best_params_["reg__regressor__alpha"]
best_score = -search.best_score_ 

print(f"Best Alpha (reg__regressor__alpha): {best_alpha}")
print(f"Best MSE: {best_score}")

results_df = pd.DataFrame(search.cv_results_)

columns_to_see = [
    'param_reg__regressor__alpha',
    'mean_test_neg_mean_squared_error',
    'std_test_neg_mean_squared_error',
    'mean_test_r2', 
    'std_test_r2'
]

columns_to_see = [col for col in columns_to_see if col in results_df.columns]

print("\n--- CV Details ---")
with pd.option_context('display.float_format', '{:,.8f}'.format):
    print(results_df[columns_to_see].sort_values(by='mean_test_neg_mean_squared_error', ascending=False))

# Save model
# joblib.dump(search.best_estimator_, 'final_ridge_pipeline.pkl')


--- GridSearchCV ... ---
Fitting 10 folds for each of 5 candidates, totalling 50 fits
Best Alpha (reg__regressor__alpha): 100000.0
Best MSE: 6.99096637740855e-06

--- CV Details ---
  param_reg__regressor__alpha  mean_test_neg_mean_squared_error  \
4            100,000.00000000                       -0.00000699   
3             10,000.00000000                       -0.00000699   
2              1,000.00000000                       -0.00000701   
1                100.00000000                       -0.00000702   
0                 10.00000000                       -0.00000704   

   std_test_neg_mean_squared_error  mean_test_r2  std_test_r2  
4                       0.00000319   -0.00026710   0.00050377  
3                       0.00000319   -0.00078645   0.00102995  
2                       0.00000319   -0.00250357   0.00255797  
1                       0.00000320   -0.00477970   0.00551090  
0                       0.00000320   -0.00687821   0.00933592  


## Grid Search CV with Sharp Ratio

In [8]:
def calculate_sharpe(pnl_series):
    if pnl_series.std() == 0:
        return 0.0
    return pnl_series.mean() / pnl_series.std()

alphas_to_test = [1.0, 5.0, 10.0, 100.0, 1000.0, 10000.0]

# --- TimeSeriesSplit ---
tscv = TimeSeriesSplit(n_splits=10)

# --- Mannual CV ---
print("--- Manual CV with Sharp ratio ---")

validation_results = {}

for alpha in alphas_to_test:
    print(f"  testing Alpha: {alpha} ...")
    
    pipe = Pipeline([
        ("imp",  SimpleImputer(strategy="median")),
        ("clip", QuantileClipper(lower=0.005, upper=0.995)),
        ("sc",   MaxAbsScaler()),
        ("reg",  TransformedTargetRegressor(
                    regressor=Ridge(solver="lsqr", alpha=alpha),
                    transformer=StandardScaler())
        ),
    ])
    
    fold_pnls = []

    for train_idx, val_idx in tscv.split(X_val):
        X_train_fold, X_val_fold = X_val.iloc[train_idx], X_val.iloc[val_idx]
        y_train_fold, y_val_fold = y_val.iloc[train_idx], y_val.iloc[val_idx]
        
        pipe.fit(X_train_fold, y_train_fold)
        
        preds = pipe.predict(X_val_fold)
        
        signals = np.sign(preds)
        
        pnl = signals * y_val_fold
        fold_pnls.append(pnl)

    validation_results[alpha] = pd.concat(fold_pnls)

# --- Compare all Alphas SR ---
print("\n--- CV Result (Sharp Ratio) ---")
final_sharpes = {}
for alpha, pnl_series in validation_results.items():
    final_sharpes[alpha] = calculate_sharpe(pnl_series)

# Result
sharpe_df = pd.Series(final_sharpes, name="Sharpe_Ratio").sort_values(ascending=False)
print(sharpe_df)

# --- Best Alpha ---
best_alpha_by_sharpe = sharpe_df.idxmax()

print(f"Best Alpha (SR): {best_alpha_by_sharpe}")
print(f"Best SR: {sharpe_df.max():.6f}")

--- Manual CV with Sharp ratio ---
  testing Alpha: 1.0 ...
  testing Alpha: 5.0 ...
  testing Alpha: 10.0 ...
  testing Alpha: 100.0 ...
  testing Alpha: 1000.0 ...
  testing Alpha: 10000.0 ...

--- CV Result (Sharp Ratio) ---
5.0        0.002012
10.0       0.001707
1.0        0.001387
100.0      0.001000
10000.0   -0.000238
1000.0    -0.000696
Name: Sharpe_Ratio, dtype: float64
Best Alpha (SR): 5.0
Best SR: 0.002012


## Fine Tune Grid Search CV with Sharp Ratio

In [9]:
alphas_to_test = np.linspace(1, 10, 10)

# --- TimeSeriesSplit ---
tscv = TimeSeriesSplit(n_splits=10)

# --- Mannual CV ---
print("--- Fine tuned Manual CV with Sharp ratio ---")

validation_results = {}

for alpha in alphas_to_test:
    print(f"  testing Alpha: {alpha} ...")
    
    pipe = Pipeline([
        ("imp",  SimpleImputer(strategy="median")),
        ("clip", QuantileClipper(lower=0.005, upper=0.995)),
        ("sc",   MaxAbsScaler()),
        ("reg",  TransformedTargetRegressor(
                    regressor=Ridge(solver="lsqr", alpha=alpha),
                    transformer=StandardScaler())
        ),
    ])
    
    fold_pnls = []

    for train_idx, val_idx in tscv.split(X_val):
        X_train_fold, X_val_fold = X_val.iloc[train_idx], X_val.iloc[val_idx]
        y_train_fold, y_val_fold = y_val.iloc[train_idx], y_val.iloc[val_idx]
        
        pipe.fit(X_train_fold, y_train_fold)
        
        preds = pipe.predict(X_val_fold)
        
        signals = np.sign(preds)
        
        pnl = signals * y_val_fold
        fold_pnls.append(pnl)

    validation_results[alpha] = pd.concat(fold_pnls)

# --- Compare all Alphas SR ---
print("\n--- CV Result (Sharp Ratio) ---")
final_sharpes = {}
for alpha, pnl_series in validation_results.items():
    final_sharpes[alpha] = calculate_sharpe(pnl_series)

# Result
sharpe_df = pd.Series(final_sharpes, name="Sharpe_Ratio").sort_values(ascending=False)
print(sharpe_df)

# --- Best Alpha ---
best_alpha_by_sharpe = sharpe_df.idxmax()

print(f"Best Alpha (SR): {best_alpha_by_sharpe}")
print(f"Best SR: {sharpe_df.max():.6f}")

--- Fine tuned Manual CV with Sharp ratio ---
  testing Alpha: 1.0 ...
  testing Alpha: 2.0 ...
  testing Alpha: 3.0 ...
  testing Alpha: 4.0 ...
  testing Alpha: 5.0 ...
  testing Alpha: 6.0 ...
  testing Alpha: 7.0 ...
  testing Alpha: 8.0 ...
  testing Alpha: 9.0 ...
  testing Alpha: 10.0 ...

--- CV Result (Sharp Ratio) ---
4.0     0.002047
5.0     0.002012
9.0     0.001939
6.0     0.001898
7.0     0.001773
8.0     0.001738
10.0    0.001707
2.0     0.001697
3.0     0.001491
1.0     0.001387
Name: Sharpe_Ratio, dtype: float64
Best Alpha (SR): 4.0
Best SR: 0.002047
